# Chargement des données et séparation des features et target

Avant tout préprocessing, on établit la séparation des données de train et de test afin de limiter la possibilité de data leak entre les jeux de données de train et de test.

In [1]:
import pandas as pd
import numpy as np

X_train = pd.read_csv("../data/X_train_update.csv", index_col = 0)
y_train = pd.read_csv("../data/Y_train_CVw08PX.csv", index_col = 0)
X_test = pd.read_csv("../data/X_test_update.csv", index_col = 0)

# Preprocessing des données

In [2]:
from scipy.sparse import hstack

# Traitement des variables textuelles
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer

# Stopwords

html_stopwords = [
    'html', 'head', 'body', 'div', 'span', 'p', 'br', 'a', 'img', 'ul', 'li', 'ol', 'table',
    'tr', 'td', 'th', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'b', 'i', 'u', 'strong', 'em',
    'eacute', 'agrave'
]
punctuation_words = [",", ".", "``", "@", "*", "(", ")", "...", "!", "?", "-", 
                  "_", ">", "<", ":", "/", "=", "--", "©", "~", ";", "\\", "\\\\"]
final_stopwords = stopwords.words('english') + stopwords.words('french') + html_stopwords + punctuation_words


# Stemming and processing

stemmer = FrenchStemmer()

def stemming(mots) :
    sortie = []
    for string in mots :
        radical = stemmer.stem(string)
        if (radical not in sortie) : sortie.append(radical)
    return sortie


def preprocessing(text, with_stemming=False):
    text = text.lower()
    tokens = word_tokenize(text)
    result = [word for word in tokens if word not in final_stopwords]
    if with_stemming:
        result = stemming(result)
    return ' '.join(result)


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/cramarokoto/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## Preprocessing des données textuelles

### Création de la variable `has_description` basée sur la présence de la variable `description`

Nous avons vu lors de l'exploration que la répartition de la présence de description n'est pas homogène parmi les catégories de produits. On peut en déduire que l'absence ou présence de description est une information à part entière qu'il peut être intéressant d'exploiter dans nos modèles.

Nous décidons de créer la variable `has_description` qui vaut :
- 1 si la description est présente
- 0 sinon

In [3]:
# Pour X_train
X_train['has_description'] = X_train['description'].notnull() & (X_train['description'].str.strip() != "")
X_train['has_description'] = X_train['has_description'].astype(int)

# Pour X_test
X_test['has_description'] = X_test['description'].notnull() & (X_test['description'].str.strip() != "")
X_test['has_description'] = X_test['has_description'].astype(int)

### Fusion des variables `designation` et `description` dans `full_description`

La `description` et la `designation` sont toutes les deux des valeurs textuelles libres. Bien que `designation` soit plus courte, elles apportent le même type d'information sur le produit c'est-à-dire sa description et son appellation commune. `description` pouvant être nulle, cela peut être gênant pour son intégration dans les modèles. 

Afin de faciliter le traitement de ces deux valeurs textuelles, nous décidons de les fusionner dans une variable nommée `full_description`

In [4]:
# Pour X_train
X_train['full_description'] = (
    X_train['designation'] + " " + X_train['description'].fillna('')
).str.strip()
# Nettoyage des colonnes inutiles
X_train = X_train.drop(columns=['designation', 'description'])

# Pour X_test
X_test['full_description'] = (
    X_test['designation'] + " " + X_test['description'].fillna('')
).str.strip()
# Nettoyage des colonnes inutiles
X_test = X_test.drop(columns=['designation', 'description'])

### Nettoyage, tokenisation et stemmatisation de la variable `full_description`

In [5]:
result = pd.DataFrame(columns=['preprocessed_full_description', 'full_description'])

for i in range(0, 10):
    current_description = X_train.loc[i, 'full_description']
    result.loc[i, 'full_description'] = current_description
    result.loc[i, 'preprocessed_full_description'] = preprocessing(current_description, with_stemming=True)

display(result)

,preprocessed_full_description,full_description
0,olivi personalisiert notizbuch 150 seiten punk...,Olivia: Personalisiertes Notizbuch / 150 Seite...
1,journal art n° 133 28/09/2001 l'art march salo...,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...
2,grand stylet ergonom bleu gamepad nintendo wii...,Grand Stylet Ergonomique Bleu Gamepad Nintendo...
3,peluch donald europ disneyland 2000 marionnet ...,Peluche Donald - Europe - Disneyland 2000 (Mar...
4,guerr tuqu luc id & grandeur veut organis jeu ...,La Guerre Des Tuques Luc a des id&eacute;es de...
5,afriqu contemporain n° 212 hiv 2004 dossi japon,Afrique Contemporaine N° 212 Hiver 2004 - Doss...
6,christof e bildungsprozessen auf der spur,Christof E: Bildungsprozessen Auf Der Spur
7,conquer sept cahi couvertur polypro 240 x 320 ...,Conquérant Sept Cahier Couverture Polypro 240 ...
8,puzzl scooby-doo post 2x35 piec,Puzzle Scooby-Doo Avec Poster 2x35 Pieces
9,tent pli v3s5-pro pvc blanc 3 x 4m50 longueur ...,Tente Pliante V3s5-Pro Pvc Blanc - 3 X 4m50 - ...


On note avec l'extrait ci-dessus que la stemmatisation est trop violente sur le contenu de la variable d'une part et qu'il n'est pas adapté sur un contenu multilingue tel que nous avons ici (français, anglais mais aussi parfois allemand). Afin de limiter la perte en information, nous décidons de ne pas appliquer de lemmatisation ni de stemmatisation.

Il aurait aussi été possible de traduire les textes pour uniformiser la langue employée mais cela représente :
- un traitement coûteux
- un risque de perte d'information si la traduction n'est pas de bonne qualité

Comme le jeu de données est conséquent, on peut s'attendre à ce que les algorithme d'analyse textuel arrivent à extraire l'information utile malgré la présence de multiple langues.

On décide donc de limiter le prétraitement textuel aux éléments suivants :
- nettoyage des stopwords anglais et français
- nettoyage de la ponctuation
- tokenisation des chaînes de caractères

In [6]:
X_train['preprocessed_full_description'] = ""
X_test['preprocessed_full_description'] = ""

for i in X_train.index:
    X_train.loc[i, 'preprocessed_full_description'] = preprocessing(X_train.loc[i, 'full_description'])

for i in X_test.index:
    X_test.loc[i, 'preprocessed_full_description'] = preprocessing(X_test.loc[i, 'full_description'])

In [7]:
display(X_train[["full_description", "preprocessed_full_description"]].head(10))
display(X_test[["full_description", "preprocessed_full_description"]].head(10))

,full_description,preprocessed_full_description
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,olivia personalisiertes notizbuch 150 seiten p...
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,journal arts n° 133 28/09/2001 l'art marche sa...
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,grand stylet ergonomique bleu gamepad nintendo...
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,peluche donald europe disneyland 2000 marionne...
4,La Guerre Des Tuques Luc a des id&eacute;es de...,guerre tuques luc id & grandeur veut organiser...
5,Afrique Contemporaine N° 212 Hiver 2004 - Doss...,afrique contemporaine n° 212 hiver 2004 dossie...
6,Christof E: Bildungsprozessen Auf Der Spur,christof e bildungsprozessen auf der spur
7,Conquérant Sept Cahier Couverture Polypro 240 ...,conquérant sept cahier couverture polypro 240 ...
8,Puzzle Scooby-Doo Avec Poster 2x35 Pieces,puzzle scooby-doo poster 2x35 pieces
9,Tente Pliante V3s5-Pro Pvc Blanc - 3 X 4m50 - ...,tente pliante v3s5-pro pvc blanc 3 x 4m50 long...


,full_description,preprocessed_full_description
84916,Folkmanis Puppets - 2732 - Marionnette Et Théâ...,folkmanis puppets 2732 marionnette théâtre min...
84917,Porte Flamme Gaxix - Flamebringer Gaxix - 136/...,porte flamme gaxix flamebringer gaxix 136/220 ...
84918,Pompe de filtration Speck Badu 95,pompe filtration speck badu 95
84919,Robot de piscine électrique <p>Ce robot de pis...,robot piscine électrique robot piscine & # 39 ...
84920,Hsm Destructeur Securio C16 Coupe Crois¿E: 4 X...,hsm destructeur securio c16 coupe crois¿e 4 x ...
84921,Cadre Universal Pro Mont Accessoires Pour Dji ...,cadre universal pro mont accessoires dji osmo ...
84922,5 Biberons Roses En Pâte À Sucre Lot de 5 bibe...,5 biberons roses pâte sucre lot 5 biberons 3cm...
84923,Maigrir Rester Jeune N°1 Octobre 1974,maigrir rester jeune n°1 octobre 1974
84924,Grand Canapé 3 Places Chesterfield Blanc Le vé...,grand canapé 3 places chesterfield blanc vérit...
84925,Piscine Beach Wave 229x229x56 cm 57495NP <p>Ce...,piscine beach wave 229x229x56 cm 57495np cette...


### Application de TF-IDF à la variable `full_description`

En l'état, les chaînes de caractères dans full_description sont difficilement exploitables par des algorithmes de machine learning.

Il est nécessaire de les transformer en valeurs numériques interprétables. Nous avons décidé d'appliquer le TF-IDF pour une extraction de donnée rapide et efficace suite à notre analyse en wordcloud qui fait apparaître la répétition de mots clés pour chaque catégorie de produits.

In [8]:

# Initialisation du vecteur TF-IDF
tfidf = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1,2)
)

# Apprentissage sur X_train et transformation
X_train_tfidf = tfidf.fit_transform(X_train['preprocessed_full_description'])
X_test_tfidf = tfidf.transform(X_test['preprocessed_full_description'])

# Suppression des colones temporaires
X_train = X_train.drop(columns=['preprocessed_full_description', 'full_description'])
X_test = X_test.drop(columns=['preprocessed_full_description', 'full_description'])

# Concaténation
X_train_final = hstack([X_train, X_train_tfidf])
X_test_final = hstack([X_test, X_test_tfidf])

### Sauvegarde des données textuelles suite à leur préprocessing

Afin de faciliter le travail sur plusieurs modèles et gagner du temps, on décide de sauvegarder les données prétraitées.

In [9]:
# Sauvegarde des données prétraitées dans des fichiers CSV

X_train.to_csv("../data/preprocessed/X_train_preprocessed.csv")
X_test.to_csv("../data/preprocessed/X_test_preprocessed.csv")

## Preprocessing des données graphiques